In [1]:
import pandas as pd
import numpy as np
from string import punctuation
from nltk.corpus import stopwords

music_review_dataframe = pd.read_json("Pet_Supplies_5.json", lines=True)
music_review_dataframe = music_review_dataframe.astype({"reviewText": 'str'}) 

music_review_dataframe["clean_reviewText"] = music_review_dataframe['reviewText'].str.replace('[^\w\s]','')

#stopwords_list = stopwords.words('english')
#music_review_dataframe["clean_reviewText"] = music_review_dataframe['clean_reviewText'].str.replace(stopwords_list,'')

In [2]:
from collections import Counter

all_text2 = ' '.join(music_review_dataframe["clean_reviewText"])
# create a list of words
words = all_text2.split()
# Count all the words using Counter Method
count_words = Counter(words)

total_words = len(words)
sorted_words = count_words.most_common(total_words)


In [3]:
vocab_to_int = {w:i+1 for i, (w,c) in enumerate(sorted_words)}
reviews_int = []
for review in music_review_dataframe["clean_reviewText"]:
    r = [vocab_to_int[w] for w in review.split()]
    reviews_int.append(r)
#reviews_int = pad_features(reviews_int, 100)
music_review_dataframe['reviews_int']=reviews_int

In [4]:
labelList=[]
for rating in music_review_dataframe["overall"]:
    if rating>=4:
        labelList.append(1)
    else:
        labelList.append(0)
music_review_dataframe['Label']=labelList

In [5]:
split_frac = 0.8
lenR = len(music_review_dataframe['reviews_int'])
train_x = music_review_dataframe['reviews_int'][0:int(split_frac*lenR)]
train_y = music_review_dataframe['Label'][0:int(split_frac*lenR)]
test_x = music_review_dataframe['reviews_int'][int(split_frac*lenR):]
test_y = music_review_dataframe['Label'][int(split_frac*lenR):]

In [6]:
#training manual
def train1(review, label, rate, total):
    if label == 1:
        for word in review:
            total[word-1] = total[word-1]+(rate)
    else:
        for word in review:
            total[word-1] = total[word-1]-1
    return total

In [7]:
#training all
numReview = 0
negative = 0
positive = 0
for label in train_y:
    if label == 0:
        negative = negative+1
    else:
        positive = positive+1
rate = negative/(positive-negative)
#rate = .3
wordScore = np.zeros(len(sorted_words))
for review in train_x:
    wordScore = train1(review, train_y[numReview], rate, wordScore)
    numReview=numReview+1
numWord=0
wordPercent=[]
for word in wordScore:
    #if numWord>15:
    wordPercent.append(word/sorted_words[numWord][1])
    #else:
     #   wordPercent.append(0)
    numWord=numWord+1

In [8]:
#testing
def predict(review, wordPercent):
    reviewTotal = 0
    for word in review:
        val = wordPercent[word-1]
        reviewTotal = reviewTotal+val
    if reviewTotal>0:
        return 1
    else:
        return 0

In [9]:
trueP = 0
falseP = 0
trueN = 0
falseN = 0
yVal = int(split_frac*lenR)
for review in test_x:
    prediction = predict(review, wordPercent)
    if test_y[yVal] ==  prediction:
        if prediction == 1:    
            trueP = trueP + 1
        elif prediction == 0:
            trueN = trueN + 1
    elif test_y[yVal] < prediction:
        #fPReview = review
        falseP = falseP + 1
    elif test_y[yVal] > prediction:
        #fNReview = review
        falseN = falseN + 1
    yVal = yVal + 1
        

In [11]:
#Used for bug fixing

#print(train_x[0])
#print(sorted_words[0:100])
#print(test_x)
print(trueP)
print(falseP)
print(trueN)
print(falseN)

314907
38557
49878
16323


In [12]:
#metrics
precision = trueP/(trueP+falseP)
recall = trueP/(trueP+falseN)
confusion_matrix = [[trueP, falseN],[falseP, trueN]]
print('Precision for positives: ',precision)
print('Recall for positives: ',recall)
print('Confusion Martix: ',confusion_matrix)
precisionN = trueN/(trueN+falseN)
recallN = trueN/(trueN+falseP)
print('Precision for negatives: ',precisionN)
print('Recall for negatives: ', recallN)

Precision for positives:  0.8909167553131295
Recall for positives:  0.9507200434743229
Confusion Martix:  [[314907, 16323], [38557, 49878]]
Precision for negatives:  0.7534327276023021
Recall for negatives:  0.5640074631084978


In [61]:
#Used for bug fixing, no need to run

#print(wordScore)
print(np.argmin(wordScore))
print(np.argmax(wordScore))
print(sorted_words[np.argmax(wordScore)-1][0])
print(sorted_words[np.argmin(wordScore)-1][0])
print(len(wordScore))
#print(wordPercent)
print(np.argmin(wordPercent))
print(np.argmax(wordPercent))
print(sorted_words[np.argmax(wordPercent)-1][0])
print(sorted_words[np.argmin(wordPercent)-1][0])
print((trueP+falseN)/len(test_y))
print(rate)
print(wordPercent[0:100])

19
1
the
was
413235
36236
14643
Whisker
Hepper
0.7892723958395387
0.332820673615025
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.04084972108403681, 0.025779215985135728, -0.068160884997827, -0.13463992420533671, 0.05090445863180386, 0.04960911148740769, 0.02591951882836101, 0.06092481699913582, -0.012684503782080086, 0.031058914110574834, 0.08078224938675191, 0.028554866580607317, 0.003315756668616973, -0.03200976488829117, 0.06180109693743063, 0.0945161114314622, 0.018033459510599273, 0.08151927388557305, 0.0004841892104523119, 0.019252894302332964, 0.01389883982642997, 0.03730522447314855, 0.06484289285580695, 0.08207736032394772, 0.0372132251950223, -0.00017405793999021412, 0.07776646125052195, 0.15626710244164127, 0.0732092612892134, 0.19755376122747526, 0.00705976395754167, -0.018982789512560088, -0.03134300548363942, 0.011308399067516649, -0.07688779830767183, 0.08275027320714998, 0.049931331267132346, 0.01285335596780791, 0.014137206778377149, 0.00713795926966813, 0.04781

In [66]:
#Used for bug fixing, no need to run

print(toWords(fPReview, sorted_words))
print(toWords(fNReview, sorted_words))
#print(wordPercent[126218])
print(predict(fNReview,wordPercent))

[('Latches', 46), ('are', 537700), ('too', 201784), ('heavy', 31018), ('for', 1424742), ('small', 181042), ('dogs', 396289), ('but', 762365), ('a', 2382463), ('solid', 13127), ('product', 295145)]
[('Love', 48424), ('it', 2355523), ('but', 762365), ('the', 3913696), ('1', 44037), ('star', 16938), ('comes', 38809), ('from', 264633), ('when', 297740), ('you', 387282), ('pull', 29042), ('it', 2355523), ('firm', 3630), ('there', 118358), ('is', 1479078), ('this', 1210305), ('long', 111359), ('flopping', 200), ('cord', 6480), ('hanging', 6274), ('off', 164580), ('your', 178567), ('dogs', 396289), ('neck', 26950), ('Which', 3442), ('means', 11581), ('it', 2355523), ('last', 86297), ('a', 2382463), ('few', 97796), ('hours', 31451), ('before', 80974), ('the', 3913696), ('straps', 9708), ('are', 537700), ('chewed', 21088), ('up', 354453), ('So', 93334), ('great', 317505), ('idea', 21906), ('but', 762365), ('needs', 29659), ('better', 120387), ('execution', 365), ('to', 2609135), ('avoid', 7055)

In [17]:
def toWords(review, sorted_words):
    inWords = []
    for word in review:
        inWords.append(sorted_words[word-1])
    return inWords